# Import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# from keras.models import Sequential
# from keras.layers import Dense
import category_encoders as ce


In [2]:
df = pd.read_csv("../datasets/final_data/FIFA Data 1990-2018.csv")
df_teams = pd.read_csv("../datasets/final_data/world_cup_comparisons.csv")
fifa_ranking = pd.read_csv("../datasets/final_data/fifa_ranking.csv")
continent = pd.read_csv("../datasets/final_data/countries and continents1.csv",encoding ='utf-8',na_filter = False)
big_df = pd.read_csv("../datasets/results.csv")

big_df=big_df.replace({"South Korea":"Korea Republic"})
big_df = big_df.replace({"Iran":"IR Iran"})
big_df = big_df.replace({"Soviet Union":"Russia"})
big_df = big_df.replace({"Czechoslovakia":"Czech Republic"})




df=df.replace({"South Korea":"Korea Republic"})
df = df.replace({"Iran":"IR Iran"})
df = df.replace({"Soviet Union":"Russia"})
df = df.replace({"Czechoslovakia":"Czech Republic"})


df_teams=df_teams.replace({"South Korea":"Korea Republic"})
df_teams = df_teams.replace({"Iran":"IR Iran"})

fifa_ranking=fifa_ranking.replace({"South Korea":"Korea Republic"})
fifa_ranking = fifa_ranking.replace({"Iran":"IR Iran"})


In [3]:
df =df.drop(['Winner'],axis=1)
df.head()

Team1         Team2       Rounds   Year_com  Day  Month  Year  Score1  \
0    Russia  Saudi Arabia  Group Phase  2018-6-14   14      6  2018       5   
1     Egypt       Uruguay  Group Phase  2018-6-15   15      6  2018       0   
2   Morocco       IR Iran  Group Phase  2018-6-15   15      6  2018       0   
3  Portugal         Spain  Group Phase  2018-6-15   15      6  2018       3   
4    France     Australia  Group Phase  2018-6-16   16      6  2018       2   

   Score2  Ranking T1  Score T1  Ranking T2  Score T2  
0       0        70.0     457.0        67.0     465.0  
1       1        45.0     649.0        14.0    1018.0  
2       1        41.0     686.0        37.0     708.0  
3       3         4.0    1274.0        10.0    1126.0  
4       1         7.0    1198.0        36.0     718.0

In [4]:
big_df.head()

date home_team away_team  home_score  away_score tournament     city  \
0  1872-11-30  Scotland   England           0           0   Friendly  Glasgow   
1  1873-03-08   England  Scotland           4           2   Friendly   London   
2  1874-03-07  Scotland   England           2           1   Friendly  Glasgow   
3  1875-03-06   England  Scotland           2           2   Friendly   London   
4  1876-03-04  Scotland   England           3           0   Friendly  Glasgow   

    country  
0  Scotland  
1   England  
2  Scotland  
3   England  
4  Scotland

In [5]:
#Adding who is the winner 
winner_big = []
for i in range (len(big_df['home_team'])):
    if big_df['home_score'][i] > big_df['away_score'][i]:
        winner_big.append(big_df['home_team'][i])
        
    elif big_df['home_score'][i] < big_df['away_score'][i]:
        winner_big.append(big_df['away_team'][i])
        
    else:
        winner_big.append('Draw')


big_df['winner'] = winner_big
big_df.head()

date home_team away_team  home_score  away_score tournament     city  \
0  1872-11-30  Scotland   England           0           0   Friendly  Glasgow   
1  1873-03-08   England  Scotland           4           2   Friendly   London   
2  1874-03-07  Scotland   England           2           1   Friendly  Glasgow   
3  1875-03-06   England  Scotland           2           2   Friendly   London   
4  1876-03-04  Scotland   England           3           0   Friendly  Glasgow   

    country    winner  
0  Scotland      Draw  
1   England   England  
2  Scotland  Scotland  
3   England      Draw  
4  Scotland  Scotland

In [6]:
df.head()

Team1         Team2       Rounds   Year_com  Day  Month  Year  Score1  \
0    Russia  Saudi Arabia  Group Phase  2018-6-14   14      6  2018       5   
1     Egypt       Uruguay  Group Phase  2018-6-15   15      6  2018       0   
2   Morocco       IR Iran  Group Phase  2018-6-15   15      6  2018       0   
3  Portugal         Spain  Group Phase  2018-6-15   15      6  2018       3   
4    France     Australia  Group Phase  2018-6-16   16      6  2018       2   

   Score2  Ranking T1  Score T1  Ranking T2  Score T2  
0       0        70.0     457.0        67.0     465.0  
1       1        45.0     649.0        14.0    1018.0  
2       1        41.0     686.0        37.0     708.0  
3       3         4.0    1274.0        10.0    1126.0  
4       1         7.0    1198.0        36.0     718.0

# Duplicate

In [7]:

# we should change all respective column for duplicate
for index,row in df.iterrows():
    df.set_value(index+488,["Team1","Team2","Rounds","Year_com",'Day','Month','Year',"Score1","Score2",'Ranking T1','Score T1','Ranking T2','Score T2'],[row["Team2"],row["Team1"],row["Rounds"],row['Year_com'],row['Day'],row['Month'],row["Year"],row["Score2"],row["Score1"],row['Ranking T2'],row['Score T2'],row['Ranking T1'],row['Score T1']])
df.shape


/home/milad/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


(976, 13)

In [8]:
df.iloc[0]

Team1               Russia
Team2         Saudi Arabia
Rounds         Group Phase
Year_com         2018-6-14
Day                     14
Month                    6
Year                  2018
Score1                   5
Score2                   0
Ranking T1              70
Score T1               457
Ranking T2              67
Score T2               465
Name: 0, dtype: object

In [9]:
df.iloc[488]

Team1         Saudi Arabia
Team2               Russia
Rounds         Group Phase
Year_com         2018-6-14
Day                     14
Month                    6
Year                  2018
Score1                   0
Score2                   5
Ranking T1              67
Score T1               465
Ranking T2              70
Score T2               457
Name: 488, dtype: object

In [10]:
def data_collection(n,Team,Year_complete,big_df):
    temp = big_df[((big_df['home_team'] == Team)|(big_df['away_team'] == Team))&(big_df['date']< Year_complete)&(big_df['tournament'] !='FIFA World Cup')].tail(n)
    number_of_winning = temp[temp['winner'] == Team].count()[0]
    number_of_failure = temp[((temp['winner'] != Team) & (temp['winner'] != 'Draw'))].count()[0]
    number_of_draw =    temp[temp['winner'] == 'Draw'].count()[0]
    print(Team," ",number_of_winning," ",number_of_failure," ",number_of_draw)
    return number_of_winning,number_of_failure,number_of_draw


In [11]:
NW1_1 =[]
NF1_1= []
ND1_1= []

NW1_2 =[]
NF1_2= []
ND1_2= []

for i in range (len(df['Team1'])):
    NW1,NF1,ND1 = data_collection(100,df['Team1'][i],df['Year_com'][i],big_df)
    NW2,NF2,ND2 = data_collection(100,df['Team2'][i],df['Year_com'][i],big_df)
    
    NW1_1.append(NW1)
    NF1_1.append(NF1)
    ND1_1.append(ND1)
    
    NW1_2.append(NW2)
    NF1_2.append(NF2)
    ND1_2.append(ND2)
    
df['NW1'] = NW1_1 
df['NF1'] = NF1_1  
df['ND1'] = ND1_1

df['NW2'] = NW1_2 
df['NF2'] = NF1_2  
df['ND2'] = ND1_2

Russia   48   25   27
Saudi Arabia   41   36   23
Egypt   62   24   14
Uruguay   46   26   28
Morocco   43   26   31
IR Iran   63   14   23
Portugal   62   18   20
Spain   72   13   15
France   59   20   21
Australia   50   26   24
Argentina   60   16   24
Iceland   40   41   19
Peru   39   36   25
Denmark   42   31   27
Croatia   58   18   24
Nigeria   48   20   32
Costa Rica   40   32   28
Serbia   47   34   19
Germany   64   15   21
Mexico   55   22   23
Brazil   71   11   18
Switzerland   54   24   22
Sweden   53   28   19
Korea Republic   53   27   20
Belgium   56   22   22
Panama   40   30   30
Tunisia   42   25   33
England   62   13   25
Colombia   48   26   26
Japan   57   21   22
Poland   46   24   30
Senegal   47   22   31
Russia   48   25   27
Egypt   62   24   14
Portugal   62   18   20
Morocco   43   26   31
Uruguay   46   26   28
Saudi Arabia   41   36   23
IR Iran   63   14   23
Spain   72   13   15
Denmark   42   31   27
Australia   50   26   24
France   59   20   21
P

Ghana   41   32   27
Germany   62   17   21
England   61   19   20
Argentina   62   20   18
Mexico   57   29   14
Netherlands   65   12   23
Slovakia   37   38   25
Brazil   65   11   24
Chile   46   30   24
Paraguay   38   32   30
Japan   54   22   24
Spain   75   8   17
Portugal   58   15   27
Netherlands   65   12   23
Brazil   65   11   24
Uruguay   43   28   29
Ghana   41   32   27
Argentina   62   20   18
Germany   62   17   21
Paraguay   38   32   30
Spain   75   8   17
Uruguay   43   28   29
Netherlands   65   12   23
Germany   62   17   21
Spain   75   8   17
Uruguay   43   28   29
Germany   62   17   21
Netherlands   65   12   23
Spain   75   8   17
Germany   53   24   23
Costa Rica   45   35   20
Poland   53   27   20
Ecuador   36   39   25
England   53   19   28
Paraguay   39   30   31
Trinidad and Tobago   42   42   16
Sweden   44   22   34
Argentina   60   18   22
Côte dIvoire   0   0   0
Serbia and Montenegro   0   0   0
Netherlands   56   11   33
Mexico   52   28   20
I

Cameroon   40   27   33
Scotland   44   30   26
Morocco   51   18   31
Brazil   69   9   22
Norway   48   20   32
France   55   13   32
Denmark   50   23   27
South Africa   45   27   19
Saudi Arabia   52   24   24
Spain   56   18   26
Bulgaria   41   34   25
Nigeria   44   28   28
Paraguay   44   32   24
Belgium   45   28   27
Korea Republic   48   22   30
Netherlands   57   20   23
Mexico   45   25   30
Germany   64   12   24
IR Iran   49   23   28
USA   41   32   27
Yugoslavia   54   28   18
Japan   50   27   23
Jamaica   48   24   28
Argentina   49   17   34
Croatia   39   14   17
Romania   53   24   23
Tunisia   48   23   29
Colombia   42   23   35
England   51   14   35
Italy   57   15   28
Norway   48   20   32
Brazil   69   9   22
Chile   43   33   24
France   55   13   32
Paraguay   44   32   24
Nigeria   44   28   28
Denmark   50   23   27
Germany   64   12   24
Mexico   45   25   30
Netherlands   57   20   23
Yugoslavia   54   28   18
Romania   53   24   23
Croatia   39   14

Iceland   40   41   19
Germany   64   15   21
Korea Republic   53   27   20
Sweden   53   28   19
Mexico   55   22   23
Brazil   71   11   18
Serbia   47   34   19
Costa Rica   40   32   28
Switzerland   54   24   22
Poland   46   24   30
Japan   57   21   22
Colombia   48   26   26
Senegal   47   22   31
Tunisia   42   25   33
Panama   40   30   30
Belgium   56   22   22
England   62   13   25
Argentina   60   16   24
France   59   20   21
Portugal   62   18   20
Uruguay   46   26   28
Russia   48   25   27
Spain   72   13   15
Denmark   42   31   27
Croatia   58   18   24
Mexico   55   22   23
Brazil   71   11   18
Japan   57   21   22
Belgium   56   22   22
Switzerland   54   24   22
Sweden   53   28   19
England   62   13   25
Colombia   48   26   26
France   59   20   21
Uruguay   46   26   28
Belgium   56   22   22
Brazil   71   11   18
England   62   13   25
Sweden   53   28   19
Croatia   58   18   24
Russia   48   25   27
Belgium   56   22   22
France   59   20   21
England   

Ghana   40   34   26
Czech Republic   65   19   16
USA   58   22   20
Italy   51   16   33
Croatia   47   19   34
Japan   53   25   22
Australia   61   23   16
Brazil   54   19   27
Korea Republic   42   28   30
France   69   10   21
Switzerland   37   34   29
Togo   35   43   22
Ukraine   40   26   34
Saudi Arabia   55   24   21
Tunisia   49   21   30
Spain   62   14   24
Germany   53   24   23
Ecuador   36   39   25
Poland   53   27   20
Costa Rica   45   35   20
England   53   19   28
Sweden   44   22   34
Trinidad and Tobago   42   42   16
Paraguay   39   30   31
Angola   40   27   33
IR Iran   62   17   21
Mexico   52   28   20
Portugal   62   14   24
Argentina   60   18   22
Netherlands   56   11   33
Serbia and Montenegro   0   0   0
Côte dIvoire   0   0   0
Italy   51   16   33
Czech Republic   65   19   16
USA   58   22   20
Ghana   40   34   26
Brazil   54   19   27
Japan   53   25   22
Australia   61   23   16
Croatia   47   19   34
Tunisia   49   21   30
Ukraine   40   26  

Cameroon   46   21   33
Saudi Arabia   46   27   27
Netherlands   55   21   24
Russia   49   20   31
Brazil   57   15   28
Greece   38   34   28
Argentina   38   24   38
Bulgaria   39   33   28
Nigeria   45   23   32
Spain   54   23   23
Germany   56   18   26
Switzerland   42   30   28
Romania   49   24   27
Colombia   47   20   33
USA   32   38   30
Norway   38   31   31
Italy   56   18   26
Bolivia   22   50   28
Korea Republic   53   16   31
Republic of Ireland   0   0   0
Mexico   52   20   28
Russia   49   20   31
Sweden   54   22   24
Cameroon   46   21   33
Brazil   57   15   28
Netherlands   55   21   24
Belgium   45   30   25
Morocco   39   24   37
Saudi Arabia   46   27   27
Nigeria   45   23   32
Argentina   38   24   38
Greece   38   34   28
Bulgaria   39   33   28
Romania   49   24   27
USA   32   38   30
Colombia   47   20   33
Switzerland   42   30   28
Korea Republic   53   16   31
Germany   56   18   26
Spain   54   23   23
Bolivia   22   50   28
Norway   38   31   31

# Team Feature



In [ ]:
features=[]
for key in df_teams.keys():
    features.append(str(key))

features

del features[0:4]

features


In [ ]:
df_teams_avg = df_teams.groupby(['Year','team'])['goals_z'].sum()/df_teams.groupby(['Year','team'])['goals_z'].size()
df_teams_avg = df_teams_avg.to_frame()

df_teams_avg['Year'] = df_teams_avg.index.get_level_values(0).tolist()
df_teams_avg['team'] = df_teams_avg.index.get_level_values(1).tolist()


df_teams_avg.head()


In [ ]:
def avg_team(feature,df_teams_avg):
    a=df_teams.groupby(['Year','team'])[feature].sum()/df_teams.groupby(['Year','team'])[feature].size()
    b=a.to_frame()
    df_teams_avg[feature]= b[feature]
    

for feature in features:
    avg_team(feature,df_teams_avg)

df_teams_avg.head()

In [ ]:
new_df = pd.merge(df, df_teams_avg,  how='left', left_on=['Team1','Year'], right_on = ['team','Year'])
new_df1 = pd.merge(new_df,df_teams_avg ,  how='left', left_on=['Team2','Year'], right_on = ['team','Year'])


new_df1.head()

In [ ]:
df = new_df1
df.isnull().sum()

In [ ]:
df = df.dropna(axis=0, subset=['goals_z_x','goals_z_y'])
df.isnull().sum()

In [ ]:
df.head()

# Add lenth of name for each country

In [ ]:
df.to_csv("./temp.csv")

In [ ]:
df = pd.read_csv("./temp.csv")

In [ ]:
len_Team1 =[]
len_Team2 =[]

for i in range (len(df['Team1'])):
    len_Team1.append(len(df['Team1'][i]))
    len_Team2.append(len(df['Team2'][i]))
    
df['len_Team1'] =len_Team1
df['len_Team2'] = len_Team2

df.head()
    

# Add the Continent

In [ ]:
continent = continent.drop_duplicates()
df['Team1'] = df['Team1'].str.strip()
df['Team2'] = df['Team2'].str.strip()

continent['name'] = continent['name'].str.strip()
continent['Continent'] = continent['Continent'].str.strip()

In [ ]:
df.loc[df.Year >= 1990,'continent_Team1'] = df['Team1'].map(continent.set_index('name')['Continent'])
df.loc[df.Year >= 1990,'continent_Team2'] = df['Team2'].map(continent.set_index('name')['Continent'])

In [ ]:
df.head()

In [ ]:
# df["Rounds"] = df["Rounds"].cat.codes
# df["Rounds"] = df["Rounds"].astype('category')


In [ ]:
# AF = Africa
# AS = Asia
# EU = Europe
# NA = North America
# SA = South America
# OC = Oceania
# AN = Antarctica

continent_dict = {"AF":1, "AS":2, "EU":3, "OC":4, "NA":5, "SA":6, "AN":7}
continent_encoded_team1 = []
continent_encoded_team2 = []


for i in range(len(df.continent_Team1)):
    continent_encoded_team1.append(continent_dict[list(df.continent_Team1)[i]])
    continent_encoded_team2.append(continent_dict[list(df.continent_Team2)[i]])
    
df["continent_en_team1"] = continent_encoded_team1
df["continent_en_team2"] = continent_encoded_team2

df = df.drop(['continent_Team1','continent_Team2'], axis =1)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

# Add winner

In [ ]:
#Adding who is the winner 
winner = []
Winner = []
for i in range (len(df['Team1'])):
    if df['Score1'][i] > df['Score2'][i]:
        winner.append(df['Team1'][i])
        Winner.append(1)
    elif df['Score1'][i] < df['Score2'][i]:
        winner.append(df['Team2'][i])
        Winner.append(2)
    else:
        winner.append('-1')
        Winner.append(0)

df['winner'] = winner
df['Winner'] = Winner
df.head()

In [ ]:
winner_counter = []
team1_winning=[]
team2_winning=[]

team1_winning_complete=[]
team2_winning_complete=[]


for i in range(len(df['Team1'])):
    if i != 0: 
        if df['Year'][i] != df['Year'][i-1]:
            for x in team1_winning:
                team1_winning_complete.append(x)
            for y in team2_winning:
                team2_winning_complete.append(y)


            winner_counter = []
            team1_winning=[]
            team2_winning=[]
            
    
        if ((df['winner'][i] != -1) & (df['winner'][i] == df['Team1'][i])):

            team2_winning.append(winner_counter.count(df['Team2'][i]))
            team1_winning.append(winner_counter.count(df['Team1'][i]))

            winner_counter.append(df['Team1'][i])

        elif((df['winner'][i] != -1) & (df['winner'][i] == df['Team2'][i])):

            team1_winning.append(winner_counter.count(df['Team1'][i]))
            team2_winning.append( winner_counter.count(df['Team2'][i]))


            winner_counter.append(df['Team2'][i])

        else:
            team1_winning.append(winner_counter.count(df['Team1'][i]))
            team2_winning.append(winner_counter.count(df['Team2'][i]))

    else:
        team1_winning.append(0)
        team2_winning.append(0)

        

for x in team1_winning:
    team1_winning_complete.append(x)
for y in team2_winning:
    team2_winning_complete.append(y)

   
    
            
df['team1_winning'] = team1_winning_complete
df['team2_winning'] = team2_winning_complete
        
        

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.to_csv("./v1.csv")

In [ ]:
# df = pd.read_csv("./v1.csv")
df = df.drop(['Unnamed: 0'], axis= 1)
# df.head()

# Encode Country

In [ ]:
countries = list(set(df["Team1"]).union(set(df["Team2"])))
countries_dict = {}
for i in range(len(countries)):
    countries_dict[countries[i]] = i
    
Team1_encoded = []
Team2_encoded = []

for i in range(len(df)):
    Team1_encoded.append(countries_dict[list(df.Team1)[i]])
    Team2_encoded.append(countries_dict[list(df.Team2)[i]])
    
df["Team1"] = Team1_encoded
df["Team2"] = Team2_encoded

In [ ]:
df.head()

In [ ]:
df = df.drop(['Unnamed: 0','Score1','Score2','winner'], axis= 1)
df.head()

In [ ]:
rounds = df.Rounds
rounds.unique()
rounds_dict = {"Group Phase":1, "Round of 16":2, "Quarter Finals":3, "Semi Finals":4, "Play-off for third place":5, "Finals":6}
rounds_encoded = []

for i in range(len(df)):
    rounds_encoded.append(rounds_dict[list(df.Rounds)[i]])
    
df["Rounds"] = rounds_encoded

In [ ]:
df = df.drop(['team_x','team_y','Rounds'],axis=1)

In [ ]:
df = df.drop(['Team1','Team2'],axis=1)

In [ ]:
df = df.drop(['team1_winning','team2_winning'],axis=1)

In [ ]:
df = df.drop(['Score1','Score2','winner'], axis =1)
# df = df.drop(['Unnamed: 0','Score1','Score2','team_x','team_y','winner'], axis =1)

In [ ]:
df.head()

In [ ]:
df = df.drop(['len_Team1','len_Team2'], axis =1)

In [ ]:
import category_encoders as ce
df =ce.OneHotEncoder(cols=['Team1','Team2','Rounds','continent_Team1','continent_Team2']).fit_transform(df,df['Winner'])


In [ ]:
df.head()

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
# y = df['Winner']
# df = df.drop(['Winner'], axis =1)
# header_name =list(X_train.columns.values)
# print(header_name)
sfs1 = SFS(neigh, 
           k_features=10, 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=0)

sfs1 = sfs1.fit(X_train,y_train)

In [ ]:
sfs1.k_feature_idx_

In [ ]:
for i in list(sfs1.k_feature_idx_):
    print(header_name[i])

In [ ]:
df = df[['Year','Team1_10','Team1_29','Team2_10','Team2_29','Rounds_5','goals_z_x','progruns_z_y','blocks_z_y','fouled_z_y','team1_winning','Winner']]

In [ ]:
df.head()

In [ ]:
sfs1.k_score_

In [ ]:
df = df.drop(['Year_com','Day','Month'],axis=1)

In [ ]:
df = df.drop(['continent_en_team1','continent_en_team2'],axis=1)

In [ ]:
df = df.drop(['Rounds'],axis=1)

In [ ]:
for c in df.keys():
    print(c)

# Train & test

In [ ]:
X_train = df.loc[(df['Year']!= 2018)]
X_test  = df.loc[(df['Year'] == 2018)]

y_train =  X_train["Winner"]
y_test  =  X_test["Winner"]

X_train = X_train.drop(['Winner'],axis=1)
X_test = X_test.drop(['Winner'],axis=1)

# X_train = X_train.drop(['Winner','team_x','team_y'],axis=1)
# X_test = X_test.drop(['Winner','team_x','team_y'],axis=1)

In [ ]:
for c in X_train.keys():
    print(c)

In [ ]:
X_train.head()

In [ ]:
X_train = X_train.drop(['Ranking T1','Ranking T2'],axis=1)
X_test = X_test.drop(['Ranking T1','Ranking T2'],axis=1)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
X_test.iloc[0]

In [ ]:
X_test.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()  
X_train = sc.fit_transform(X_train)  
X_test = sc.transform(X_test) 


import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

import xgboost as xgb
from sklearn.datasets import dump_svmlight_file

dump_svmlight_file(X_train, y_train, 'dtrain.svm', zero_based=True)
dump_svmlight_file(X_test, y_test, 'dtest.svm', zero_based=True)
dtrain_svm = xgb.DMatrix('dtrain.svm')
dtest_svm = xgb.DMatrix('dtest.svm')


In [ ]:
param = {
    'max_depth': 50,  # the maximum depth of each tree
    'eta': 0.005,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 69}  # the number of classes that exist in this datset
num_round = 100  # the number of training iterations

In [ ]:
bst = xgb.train(param, dtrain, num_round)
preds = bst.predict(dtest)

import numpy as np
best_preds = np.asarray([np.argmax(line) for line in preds])


from sklearn.metrics import precision_score

print(accuracy_score(y_test, best_preds))

In [ ]:
{'C': 3.4067106021701203, 'gamma': 0.33159433754757905}

In [ ]:
from sklearn import svm
model_svm = svm.SVC()
model_svm.fit(X_train,y_train)

y_pred_svm = model_svm.predict(X_test)


accuracy_score(y_test, y_pred_svm)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, y_train) 
y_pred_neigh = neigh.predict(X_test)
accuracy_score(y_test, y_pred_neigh)

In [ ]:
model_svm.predict([[-1.88311701,  1.73369565,  0.29386772, -0.49299277,  1.96755193,
       -0.18129363, -0.16239592,  1.49092911, -0.63904823, -0.16016975,
       -0.35151564, -1.05590009, -0.25826668, -0.81182324, -0.57273967,
       -1.18318577, -0.40759521,  1.19900457, -0.49516943,  2.90417344,
        1.78662061, -0.74060623, -0.03438443, -1.12279921, -0.47668739,
       -0.08327925, -1.63341844,  0.7866811 , -0.28311076, -0.6396246 ,
        0.20268982,  1.42660544,  0.32660141, -1.22817515,  0.89398111,
       -2.10535758, -1.532413  ,  0.19244285, -1.21486428, -0.56510533,
        1.66210323, -0.27490465, -0.90994275, -0.74932094, -0.74932094]])

In [ ]:
model_svm.predict([[1.73369565,-1.88311701, 0.29386772, -0.49299277, 
       1.96755193,-0.18129363,  -0.03438443, -1.12279921, -0.47668739,
       -0.08327925, -1.63341844,  0.7866811 , -0.28311076, -0.6396246 ,
        0.20268982,  1.42660544,-0.16239592,  1.49092911, -0.63904823, -0.16016975,
       -0.35151564, -1.05590009, -0.25826668, -0.81182324, -0.57273967,
       -1.18318577, -0.40759521,  1.19900457, -0.49516943,  2.90417344,
        1.78662061, -0.74060623,-1.22817515,0.32660141,0.89398111,
       -2.10535758, -1.532413  ,  0.19244285, -1.21486428, -0.56510533,
        1.66210323,-0.90994275,-0.27490465,-0.74932094, -0.74932094]])

In [ ]:
for ii in range(len(y_pred_svm)):
    if y_pred_svm[ii] != y_test[ii]:
        print(y_pred_svm[ii],"  ",y_test[ii])

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

model= DecisionTreeClassifier()
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

accuracy_score(y_test, y_pred)

In [ ]:
model = Sequential()
model.add(Dense(12, input_dim=X_train.shape[1], activation='sigmoid'))
model.add(Dense(8, activation='sigmoid'))
#model.dropout()
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) #categorical crossentropy?

hist = model.fit(X_train, y_train, validation_data=(X_test,y_test), shuffle=True, epochs=100, batch_size=2)

In [ ]:
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

model_inf = ExtraTreesClassifier()
model_inf.fit(X_train, y_train)

model_tree= DecisionTreeClassifier()
model_tree.fit(X_train,y_train)

y_pred_tree = model_tree.predict(X_test)

print(accuracy_score(y_test, y_pred_tree))


information_gain = {}

for i in range(len(model_inf.feature_importances_)):
    information_gain.update({i: model_inf.feature_importances_[i]})
    
    
for key in information_gain: 
    print(key,information_gain[key])

In [ ]:
X_train